In [60]:
wdir = '/home/hoseung/Work/data/05427/'
cluster_name = wdir.split('/')[-2]

frefine= 'refine_params.txt'
fnml = 'cosmo_200.nml'
nout_ini=186
nout_fi=187
nouts = range(nout_ini,nout_fi+1)

scale = 5
npix = 800

m_halo_min = 5e9

# data loading parameters
ptype=["star pos mass"]
refine_params = True
dmo=False
draw=True
draw_halos=True
draw_part = True
draw_hydro = False
if draw_hydro:
    lmax=19
    
import load
import utils.sampling as smp
import tree.halomodule as hmo
import utils.match as mtc
import draw
import pickle
import numpy as np
from tree import tmtree

info = load.info.Info(nout_fi, wdir, load=True)
tt = tmtree.load(work_dir=wdir, filename="GalaxyMaker/TMtree.fits")

# Galaxy ID list
hh = hmo.Halo(base=wdir, nout=nout_fi, halofinder='HM', info=info, load=True, is_gal=True)
i_center = np.where(hh.data['np'] == max(hh.data['np']))
i_satellites = smp.extract_halos_within(hh.data, i_center, scale=scale)
print("Total {0} halos \n{1} halos are selected".format(len(i_satellites),sum(i_satellites)))

large_enugh = hh.data['mvir'] > m_halo_min
halo_list = hh.data['id'][i_satellites * large_enugh]
h_ind_ok, gal_ok = tmtree.check_tree_complete(tt, 0, nout_fi - nout_ini, halo_list[4:5])
final_gal = gal_ok[:,0]
ngals = len(final_gal)
print("Ngal", final_gal)

Total 475 halos 
291 halos are selected
Ngal [7]


In [ ]:
import matplotlib.pyplot as plt
import tree
from draw import pp

#for igal, gal in enumerate(final_gal):
#    for inout, nout in enumerate(nouts[1:]):
igal = 0
gal = 7
nout - 187
inout = 0
snout = str(nout).zfill(3)

# Only selected galaxy
hh = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True, is_gal=True)
hind = np.where(hh.data['id'] == halo_ok[igal,inout])[0]
h = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info)
h.derive_from(hh, hind)
print("Final target galaxy: ")
print(" ID {} \n POS {} {} {} \n Rvir {}".format(h.data['id'],
                                                h.data['x'], h.data['y'], h.data['z'],
                                                h.data['rvir']*200000))

region = smp.set_region(xc=h.data['x'],
                yc=h.data['y'],
                zc=h.data['z'],
                radius = h.data['rvir'] * 5)
s = load.sim.Sim(nout, wdir, dmo=dmo, ranges=region["ranges"], setup=True)
imgs = draw.img_obj.MapSet(info=s.info, region=region)
imgp = draw.img_obj.MapImg(info=s.info, proj='z', npix=npix, ptype=ptype)
imgp.set_region(region)

#%%
if draw_part:
    s.add_part(ptype)
    s.part.load()
    part = getattr(s.part, s.part.pt[0])

    x = part['x']
    y = part['y']
    z = part['z']
    m = part['m'] * s.info.msun # part must be normalized already!
    #imgp.set_data(draw.pp.den2d(x, y, z, m, npix, s.info, cic=True, norm_integer=True))
    imgp.set_data(draw.pp.den2d(x, y, z, m, npix, region=region, cic=True, norm_integer=True))
    imgs.ptden2d = imgp
#    imgp.show_data()

#%%
if draw_hydro:
    s.add_hydro()
    s.hydro.amr2cell(lmax=lmax)
    field = draw.pp.pp_cell(s.hydro.cell, npix, s.info, verbose=True)
    ptype = 'gas_den'
    imgh = draw.img_obj.MapImg(info=s.info, proj='z', npix=npix, ptype=ptype)
    imgh.set_data(field)
    imgh.set_region(region)
#    imgh.show_data()
    imgs.hydro = imgh
#%%

fdump = wdir + snout + 'map.pickle'
with open(fdump, 'wb') as f:
    pickle.dump(imgs, f)
if draw:
    if draw_part:
        imgs.ptden2d.plot_2d_den(save= wdir + cluster_name + snout +'star.png',
                                 vmax=20,vmin=15, dpi=400, show=False)
    if draw_hydro:    
        imgs.hydro.plot_2d_den(save= wdir + cluster_name +snout + 'hydro.png',
                               vmax=15,vmin=10, show=False, dpi=400)

In [70]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
snout = str(nout).zfill(3)
fin = wdir + snout + 'map.pickle'
with open(fin, 'rb') as f:
    img = pickle.load(f)

ptimg = img.ptden2d
fout = wdir + snout + "dmmap_" + ptimg.proj + ".png"
img.ptden2d.plot_2d_den(save=False, show=False, vmin=1e13, vmax=1e20, dpi=400, axes=ax1)

#s = load.sim.Sim(nout, base_dir)
info = load.info.Info(nout=nout, base=wdir, load=True)
#hall = tree.halomodule.Halo(nout=nout, base=wdir, halofinder="HM", info=info, load=True, is_gal=True)
#i_center = np.where(hall.data['np'] == max(hall.data['np']))
#h = tree.halomodule.Halo()
#h.derive_from(hh, [i_center]) 

ind = np.where(hall.data.mvir > 1e9)
h_sub = tree.halomodule.Halo()
h_sub.derive_from(hh, ind) 

#ax1.set_xlim(right=npix).
#ax1.set_ylim(top=npix)
#pp.pp_halo(h_sub, npix, region=img.ptden2d.region, axes=ax1, rscale=3, name=True)
pp.pp_halo(h_sub, npix, region=region, axes=ax1, rscale=3, name=True)

plt.show()
        #plt.savefig("aaa.png")

Are these parameters right?


In [73]:
nout

187